In [1]:
import os

os.chdir("../utils")

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.utils.data
import torchvision
import utils

from engine import train_one_epoch, evaluate
from PIL import Image
from PIL.ExifTags import TAGS
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms

In [3]:
class GlassesDataset(torch.utils.data.Dataset):
    
    def __init__(self, root, data_file, transform):
        self.root = root
        self.data = pd.read_csv(data_file, sep=';')
        self.transform = transform
        self.images = self.data["image_path"].values
        self.data_file_path = data_file
        
    def __getitem__(self, idx):
        image_path = os.path.join(self.root, self.images[idx])
        image = Image.open(image_path).convert("RGB")
        
        for orientation in TAGS.keys():
            if TAGS[orientation]=="Orientation":
                break
                
        exif = dict(image.getexif().items())

        if len(exif) != 0:
            if orientation in exif.keys():
                if exif[orientation] == 3:
                    image = image.rotate(180, expand=True)
                elif exif[orientation] == 6:
                    image = image.rotate(270, expand=True)
                elif exif[orientation] == 8:
                    image = image.rotate(90, expand=True)
                    
        box_list = self.data[self.data["image_path"]==self.images[idx]][["x_min", "y_min", "x_max", "y_max"]].values
        
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        
        object_count = len(box_list)
        
        labels = torch.ones((object_count,), dtype=torch.int64)
        
        iscrowd = torch.zeros((object_count,), dtype=torch.int64)
        
        area = (boxes[:,3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        image_id = torch.tensor([idx])
        
        target = {
            "boxes": boxes,
            "labels": labels,
            "iscrowd": iscrowd,
            "area": area,
            "image_id": image_id
        }
        
        if self.transform is not None:
            img = self.transform(image)
            
        return image, target
    
    def __len__(self):
        return len(self.data)

In [4]:
transformations = transforms.Compose([transforms.ToTensor()])

glasses_dataset = GlassesDataset("../../data/glasses-images", "../../data/glasses.csv", transformations)

torch.manual_seed(1)

indices = torch.randperm(len(glasses_dataset)).tolist()

train = torch.utils.data.Subset(glasses_dataset, indices[:100])
test = torch.utils.data.Subset(glasses_dataset, indices[100:])

train_loader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
test_loader = torch.utils.data.DataLoader(test, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)

In [5]:
def get_model(classes_count):
    
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    for param in model.parameters():
        param.requires_grad = False
        
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, classes_count)
    
    return model

In [6]:
this_device = torch.device("cuda" if torch.cuda.is_available else "cpu")

classes_count = 2
model = get_model(classes_count).to(this_device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /Users/vdrumea/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


AssertionError: Torch not compiled with CUDA enabled

In [7]:
epoch_count = 5

for epoch in range(epoch_count):
    train_one_epoch(model, optimizer, train_loader, this_device, epoch, print_freq=10)

NameError: name 'model' is not defined

In [ ]:
torch.save(model.state_dict(), "Path_To_Save")

loaded_model = get_model(classes_count=2)
loaded_model.load_state_dict(torch.load("Path_To_Model"))

In [ ]:
# Use this code to made a predition.
from PIL import ImageDraw
# Getting the image.
for idx in range(2,20):
    img, _ = test[idx]
    # Getting the object coordinates.
    label_boxes = np.array(test[idx][1]['boxes'])

    # Setting the model to eval state.
    loaded_model.eval()
    # Making the prediction.
    with torch.no_grad():
        prediction = loaded_model([img])

    # Getting an drawing the image.
    image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    draw = ImageDraw.Draw(image)

    # Drawing the real box around the object.
    for elem in range(len(label_boxes)):
        draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
                       (label_boxes[elem][2], label_boxes[elem][3])],
                      outline='green', width=3)
    # Drawing the predicted box around the object.
    for element in range(len(prediction[0]['boxes'])):
        boxes = prediction[0]['boxes'][element].cpu().numpy()                                      
        score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals=4)

        if score > 0.2:
            draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])],
                         outline='red', width=3)
            draw.text((boxes[0], boxes[1]), text=str(score))
    display(image)